## Extracting Knowledge Graph from Unstructured Text

Inspired by example https://jxnl.github.io/instructor/examples/knowledge_graph/, this example will show how to use ActionWeaver to create a knowledge graph from unstructured text by making use of OpenAI function calls.

**Step 1** We will use Pydantic Model to define structures (knowledge graph) we want to extract from the raw text.

In [2]:
from typing import List

from pydantic import BaseModel
from pydantic import Field


class Node(BaseModel):
    id: int
    label: str
    color: str

class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str = "black"

class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)


**Step 2**: Utilize the **action_from_model** function to convert the PyDantic model into an Agentkit action. Then, employ the `invoke` method to evaluate an example. By using `force=True`, you compel the LLM to execute the action.

In [4]:
import os

from agentkit.actions.factories.pydantic_model_to_action import action_from_model
from agentkit.llms.client.chat import completion

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

query = "Everything about jiu jitsu"

messages=[
        {
            "role": "user",
            "content": f"Help me understand the following by describing it as a detailed knowledge graph: {query}",
        }
    ]

kg = completion(
    messages=messages,
    model="gpt-4o",
    actions=[
        action_from_model(KnowledgeGraph, stop=True)
    ],
)

In [18]:
kg

KnowledgeGraph(nodes=[Node(id=1, label='Jiu Jitsu', color='blue'), Node(id=2, label='Origin', color='yellow'), Node(id=3, label='Techniques', color='yellow'), Node(id=4, label='Gi', color='yellow'), Node(id=5, label='No-Gi', color='yellow'), Node(id=6, label='Brazilian Jiu Jitsu', color='green'), Node(id=7, label='Japanese Jiu Jitsu', color='green'), Node(id=8, label='Positions', color='yellow'), Node(id=9, label='Mount', color='yellow'), Node(id=10, label='Guard', color='yellow'), Node(id=11, label='Side Control', color='yellow'), Node(id=12, label='Back Control', color='yellow'), Node(id=13, label='Submissions', color='yellow'), Node(id=14, label='Chokes', color='yellow'), Node(id=15, label='Joint Locks', color='yellow')], edges=[Edge(source=1, target=2, label='Origin', color='black'), Edge(source=1, target=3, label='Includes', color='black'), Edge(source=3, target=4, label='Includes', color='black'), Edge(source=3, target=5, label='Includes', color='black'), Edge(source=1, target=6,

In [5]:
# Copied from https://jxnl.github.io/instructor/examples/knowledge_graph/

from graphviz import Digraph


def visualize_knowledge_graph(kg: KnowledgeGraph):
    dot = Digraph(comment="Knowledge Graph")

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    dot.render("knowledge_graph.gv", view=True)

ModuleNotFoundError: No module named 'graphviz'

In [20]:
visualize_knowledge_graph(kg)

<img src="./figures/knowledge_graph.png">
